# Programa para generar Casos de Cursogramas utilizados para el entrenamiento, validación y prueba del Modelo de Detección de Objetos

1) Importar Librerias

In [ ]:
#@title Cargar las librerías 

import os
import os.path
import json
import math
import string
import numpy as np
import xml.etree.cElementTree as ET

from os.path import isfile, join
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from random import choice, choices, shuffle, sample
from random import randint, randrange

import csv

import copy 

print ("Librerías cargadas.")

2) Montar Drive:

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

3) Configuración:

In [ ]:
#@title Configuración de Parámetros


#@markdown Parámetros sobre Archivos a Generar:
cantidad_casos_generar_total = 500 #@param {type:"slider", min:1, max:5000, step:1}
if cantidad_casos_generar_total <= 0:
  cantidad_casos_generar_total = 1
prefijo_inicial_caso_generado =  "p" #@param {type:"string"}
valor_inicial_caso_generado = 1 #@param {type:"integer"}
if valor_inicial_caso_generado < 1:
  valor_inicial_caso_generado = 1
forzar_generar_combinaciones_parametros = True #@param {type:"boolean"}

#@markdown Parámetros generales sobre generación de símbolos en los casos:
generar_errores = False 
usar_espacio_entre_simbolos = "AZAR" #@param ["AZAR", "NORMAL", "REDUCIDO", "SIN TRANSICIONES"]
usar_config_probabilidad_simbolos = False #@param {type:"boolean"}
generar_simbolos_sin_referencias = False #@param {type:"boolean"}
usar_config_valores_tipo_referencias = True #@param {type:"boolean"}
usar_referencias_generadas = "AZAR" #@param ["AZAR", "SECUENCIAL_TIPO_REFERENCIA", "SECUENCIAL_SIMBOLO"]
generar_titulos_areas_cursograma = 'AZAR' #@param ["SI", "NO", "AZAR"]
generar_recuadro_exterior_cursograma = 'AZAR' #@param ["SI", "NO", "AZAR"]
usar_orientacion_cursograma = 'AZAR' #@param ["HORIZONTAL", "VERTICAL", "AZAR"]

#@markdown Parámetros especiales sobre generación forzada de símbolos en los casos:
forzar_decision = False #@param {type:"boolean"}
forzar_copia = False #@param {type:"boolean"}
forzar_area_externa = False #@param {type:"boolean"}
forzar_actualizacion = False #@param {type:"boolean"}
forzar_consulta_logica = False #@param {type:"boolean"}

#@markdown Parámetros sobre manejo del Drive:
forzar_drive_borrado = True #@param {type:"boolean"}
forzar_drive_actualizar = True #@param {type:"boolean"}

# ubicación de elementos en el drive
path_base = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Generador-Casos/' #@param {type:"string"}
path_simbolos = path_base + 'simbolos'
path_fuente = path_base + 'fuentes/verdana.ttf'
path_reglas = path_base + 'generador_reglas.json'
path_config = path_base + 'generador_simbolos_config.csv'
path_destino = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas/Cursogramas/Generados/' #@param {type:"string"}

#@markdown Parámetros de DEBUG
modo_DEBUG = False #@param {type:"boolean"}

print("Parámetros definidos.")

# si fuerza generar diferentes combinaciones de parámetros
# determina la cantidad de ciclos a realizar
if forzar_generar_combinaciones_parametros:
  print(" ++ se detecta activación de [forzar_generar_combinaciones_parametros] ")
  print("    por lo que ciertos parámetros se ignoran para generar los casos ++")
else:
  # chequea combinaciones de parámetros
  if (forzar_area_externa or forzar_actualizacion or forzar_consulta_logica):
    if usar_espacio_entre_simbolos == "SIN TRANSICIONES":
      print("Error: para generar área externa, actualizaciones o consultas lógicas se deben generar transiciones!")
      usar_espacio_entre_simbolos =  "REDUCIDO"
      print("       -> Se cambia el valora a : ", usar_espacio_entre_simbolos)


In [ ]:
#@title Carga la configuración de los Símbolos y Referencias

# carga la información de un archivo CSV
##model_drive_path
config_simbolos_file = path_config 
with open(config_simbolos_file, mode='r') as csvfile:
    lineasCSV = list(csv.reader(csvfile))

# procesa el archivo CSV
auxListS = []
auxListR = []
for l in lineasCSV:
  # si no se debe ignorar las líneas de comentarios
  if l[0][0] != "#":
    if l[0] == "$":
      # carga la configuración del símbolo
      auxListS.append( ( l[1], [ int(l[2]), int(l[3]), l[4] ] ) )
    elif l[0] == "!":
      # carga la configuración de referencia especial
      auxListR.append( ( l[1], l[2].split("*") ) )

# genera dicccionario de la configuración 
config_simbolos = dict(auxListS)
config_referencias = dict(auxListR)

print("> Configuración cargada de [", config_simbolos_file, "] ")
print("$) Configuración de Símbolos: ", len(auxListS))
print("  valores: nombreSimbolo + [probabilidadUsarSectorInterno, probabilidadUsarSectorExterno, tipoReferencia] ] ")
print("  ", config_simbolos)
print("!) Configuración de Referencias: ", len(auxListR))
print("  valores: tipoReferencia + [valores permitidos] ] ")
print("  ", config_referencias)

# constantes de posicion de configuraciones
config_Probab_Defecto = 50
config_posProbabUsarSectorInterno = 0
config_posProbabUsarSectorExterno = 1
config_posTipoReferencia = 2


4) Definir funciones auxiliares:

In [ ]:
#@title Funciones Generales

def determinarConfigTipoCaso(tipo_caso):
  global usar_config_probabilidad_simbolos
  global usar_config_valores_tipo_referencias, generar_simbolos_sin_referencias, usar_referencias_generadas
  global forzar_decision, forzar_copia, forzar_area_externa, forzar_actualizacion, forzar_consulta_logica
  prefijoNombreCaso = tipo_caso
  # limpia los parámetros
  generar_errores = False        
  # muestra tipo de combinación
  tipo_caso_genSimbolos = tipo_caso[1:2]   
  tipo_caso_genReferencias = tipo_caso[3:4]  
  # determina parámetros de generación de símbolos 
  if tipo_caso_genSimbolos == 'F':
    # símbolos forzados (al azar)
    tipo_forzado = tipo_caso[2:3]  
    tipo_caso_genSimbolos = tipo_caso_genSimbolos + tipo_forzado    
    tipo_caso_genReferencias = tipo_caso[4:5]
    usar_config_probabilidad_simbolos = False
    forzar_decision = False 
    forzar_copia = False 
    forzar_area_externa = False 
    forzar_actualizacion = False   
    forzar_consulta_logica = False  
    # determina el símbolo a forzar
    if tipo_forzado=='d':
      forzar_decision = True
    elif tipo_forzado=='c':
      forzar_copia = True
    elif tipo_forzado=='e':
      forzar_area_externa = True
    elif tipo_forzado=='a':
      forzar_actualizacion = True  
    elif tipo_forzado=='o':
      forzar_consulta_logica = True
    else: # cualquiera
      forzar_decision = True
      forzar_copia = True
      forzar_area_externa = True
      forzar_actualizacion = True  
      forzar_consulta_logica = True
  elif tipo_caso_genSimbolos == 'P':
    # símbolos considerando probabilidades de configuración    
    usar_config_probabilidad_simbolos = True
    forzar_decision = False 
    forzar_copia = False 
    forzar_area_externa = False 
    forzar_actualizacion = False   
    forzar_consulta_logica = False    
  else: #elif tipo_caso_genSimbolos == 'A':
    # símbolos al azar
    usar_config_probabilidad_simbolos = False
    forzar_decision = False 
    forzar_copia = False 
    forzar_area_externa = False 
    forzar_actualizacion = False  
    forzar_consulta_logica = False     
  # determina parámetros de generación de referencias de símbolos    
  if tipo_caso_genReferencias == 'V':
    # sin referencias (vacías)
    generar_simbolos_sin_referencias = True 
    usar_config_valores_tipo_referencias = False
    usar_referencias_generadas = "AZAR"       
  elif tipo_caso_genReferencias == 'S':
    # referencias por configuración secuenciales
    generar_simbolos_sin_referencias = False
    usar_config_valores_tipo_referencias = True
    usar_referencias_generadas = "SECUENCIAL_TIPO_REFERENCIA"       
  elif tipo_caso_genReferencias == 'C':
    # referencias por configuración al azar
    generar_simbolos_sin_referencias = False      
    usar_config_valores_tipo_referencias = True
    usar_referencias_generadas = "AZAR" 
  else: # elif tipo_caso_genReferencias == 'A':
    # con referencias totalmente al azar
    generar_simbolos_sin_referencias = False
    usar_config_valores_tipo_referencias = False
    usar_referencias_generadas = "AZAR" 
  print("\n> Combinación tipo ", tipo_caso, " (símbolos ", tipo_caso_genSimbolos, ", & referencias ", tipo_caso_genReferencias, "):")    
  return prefijoNombreCaso, tipo_caso_genSimbolos, tipo_caso_genReferencias


def determinaConfigTamanios(generar_titulos_areas):
  global usar_espacio_entre_simbolos, usar_orientacion_cursograma

  # parámetro para tamaños de símbolos (general)
  box_size = (100, 100)
  
  # determina tipo de espacio entre simbolos    
  if usar_espacio_entre_simbolos == "AZAR":
    rndTrans = randint(0, 9) 
    if (rndTrans < 2) and not (forzar_area_externa or forzar_actualizacion or forzar_consulta_logica):
      # nota: si se fuerzan esos símbolos es obligatorio usar transiciones
      seAgreganTransiciones = False
      reducirTransiciones = (rndTrans < 1)
    else:
      seAgreganTransiciones = True
      reducirTransiciones = (rndTrans < 6)
  elif usar_espacio_entre_simbolos == "SIN TRANSICIONES":
    seAgreganTransiciones = False
    reducirTransiciones = False
  elif usar_espacio_entre_simbolos == "REDUCIDO":
    seAgreganTransiciones = True
    reducirTransiciones = True
  else: # usar_espacio_entre_simbolos == "NORMAL":
    seAgreganTransiciones = True
    reducirTransiciones = False
  # determina largo y grosor de transiciones 
  if seAgreganTransiciones:
    if reducirTransiciones:
      # transiciones reducidas en tamaño
      sufijoNombreCaso = 'r'
      distancia_entre_simbolos = randrange(box_size[0]//6, box_size[0]*3//4, 5)
      areas_line_width = 1
      temporalidad_line_width = 1
      transicion_line_width = 2
      white_line_size = 5
    else:
      # transiciones normales
      sufijoNombreCaso = ''
      distancia_entre_simbolos = box_size[0]
      areas_line_width = 2
      temporalidad_line_width = 2
      transicion_line_width = 4
      white_line_size = 10
  else:      
      # sin transiciones
      sufijoNombreCaso = 's'
      distancia_entre_simbolos = box_size[0]//10
      areas_line_width = 1
      temporalidad_line_width = 1
      transicion_line_width = 2
      white_line_size = 5
  distancia_entre_simbolos = int( distancia_entre_simbolos )
  # determina orientación de la página del diagrama
  if usar_orientacion_cursograma == 'AZAR':
    page_orientacion_vertical = (randint(0, 5) < 4)
  else:
    page_orientacion_vertical = (usar_orientacion_cursograma =="VERTICAL")
  vertical_page_size = (794, 1123)
  if page_orientacion_vertical:
    page_size = (vertical_page_size[0], vertical_page_size[1])
  else:
    page_size = (vertical_page_size[1], vertical_page_size[0])
  posicion_limite = (int(page_size[0]/box_size[0])-1, int(1.5*page_size[1]/(distancia_entre_simbolos+box_size[1]))-2)
  if seAgreganTransiciones:
    if generar_titulos_areas:
      # si se generan titulos, pierde un renglón
      posicion_limite = (posicion_limite[0], posicion_limite[1]-1)
  else:
    # ajusta posicion limite por redondeo
    posicion_limite = (posicion_limite[0], posicion_limite[1]-2)
  return sufijoNombreCaso, page_size, seAgreganTransiciones, reducirTransiciones, \
   distancia_entre_simbolos, box_size, posicion_limite, \
   areas_line_width, temporalidad_line_width, transicion_line_width, white_line_size


def determinaOtraConfig():
  global generar_recuadro_exterior_cursograma, generar_titulos_areas_cursograma
  # determina recuadro cursograma
  if generar_recuadro_exterior_cursograma == "AZAR":
    generar_recuadro_exterior  = (randint(0, 5) < 3)
  else:
    generar_recuadro_exterior = (generar_recuadro_exterior_cursograma=="SI")
  # determina títulos cursgramas
  if generar_titulos_areas_cursograma  == "AZAR":
    generar_titulos_areas = (randint(0, 5) < 3)
  else:
    generar_titulos_areas = (generar_titulos_areas_cursograma =="SI")
  return generar_recuadro_exterior, generar_titulos_areas


def inicializar_carpeta_destino(path_destino, forzar_drive_borrado = False):
  if os.path.isdir(path_destino):
    if forzar_drive_borrado:
      files = os.listdir(path_destino)
      for file in files:
        os.remove(path_destino + file)
    else:
      contar_cantidad_casos(path_destino)
  else:
    os.makedirs(path_destino)
  return 


def cargar_reglas():
  with open(path_reglas) as f:
    reglas = json.load(f)
  return reglas


def crear_cursograma(nombre, page_size):
  xml_annotation = generar_documento_xml(nombre)
  page = Image.new('RGB', page_size, (255, 255, 255))
  return page, xml_annotation


def guardar_cursograma(nombre, page, xml_annotation):
  page.save(path_destino + nombre + ".png")
  xml_document = ET.ElementTree(xml_annotation)
  xml_document.write(path_destino + nombre + ".xml" )


def obtener_simbolos(box_size, reducirTransiciones):
  # obtiene las imagenes de los símbolos
  dicc = {}
  simbolosDisp = os.listdir(path_simbolos)
  for s in simbolosDisp:
    if s.find(".png") > 0 or s.find(".jpg") > 0:
      img_simbolo = Image.open(join(path_simbolos, s))
      # controla que el tamaño sea menor que el boxsize (sino lo ajusta)
      auxS0 = img_simbolo.size[0]
      auxS1 = img_simbolo.size[1]
      if reducirTransiciones:
        # se ajusta tamaño de algunos símbolos usando porcentajes predefinidos
        if "destruccion" in s or "demora" in s or "consulta" in s:
          porc = 0.5
        elif "decision" in s or "control" in s or "operacion" in s:
          porc = 0.6
        elif "conector" in s or "actualizacion" in s :
          porc = 0.70
        elif "archivo" in s:
          porc = 0.8
        elif "documento" in s:
          porc = 0.9
        elif "proceso" in s:
          porc = 0.95         
        else:
          porc = 1.0
        auxS0 = int(float(auxS0) * porc)
        auxS1 = int(float(auxS1) * porc) 
        if auxS0 <= 0:
          auxS0 = 1
        if auxS1 <= 0:
          auxS1 = 1                   
      # si hace falta se ajusta el tamaño para que entre en box_size
      if box_size[0] < auxS0:
        auxS1 = int(float(auxS1 * box_size[0]) / auxS0)
        auxS0 = box_size[0]       
      if box_size[1] < auxS1:
        auxS0 = int(float(auxS0 * box_size[1]) / auxS1)
        auxS1 = box_size[1]    
      if (auxS0 <= 0 or auxS1 <= 0):
        print("!! Error en obtener_simbolos: ", s, "tamaño actual: ", img_simbolo.size, " nuevo tamaño [", auxS0, ",", "]!!")
      # si cambio, ajusta el tamaño de la imagen
      if auxS0 > 0 and auxS1 > 0 and (auxS0 != img_simbolo.size[0] or auxS1 != img_simbolo.size[1]):
        img_simbolo = img_simbolo.resize((auxS0,auxS1), Image.ANTIALIAS)
      # guarda el tamaño y la imagen del símbolo
      dicc[s] = [(auxS0, auxS1), img_simbolo]
  return dicc



# función especial para probar como se muestran los símbolos y sus referencias
def debug_probarSimbolos():
    global page, xml_annotation, dicc_Simbolos, page_size, diccReferenciasSecuenciales
    global usar_config_valores_tipo_referencias, usar_referencias_generadas, seAgreganTransiciones
    global reducirTransiciones, generar_simbolos_sin_referencias
    # config común
    usar_config_valores_tipo_referencias = False
    usar_referencias_generadas = "AZAR"    
    seAgreganTransiciones = True
    page_size = (1000, 15000)
    diccReferenciasSecuenciales = {}
    # define lienzo
    page, xml_annotation = crear_cursograma("prueba", page_size)
    coord = (300, 10)      
    font = ImageFont.truetype(font=path_fuente, size=14)
    draw = ImageDraw.Draw(page)
    # procesa cada combinaciín de configuraciones
    for conf in [ (True, False), (True, True), (False, False), (False, True) ]:
        generar_simbolos_sin_referencias = conf[0]
        reducirTransiciones = reducirTransiciones = conf[1]
        if reducirTransiciones:
          text = "Tamaño Reducido"
        else:
          text = "Tamaño Normal"
        if generar_simbolos_sin_referencias:
          text = text + " - Sin Referencias"
        else:
          text = text + " - Con Referencias"
        coord = (coord[0], coord[1]+20)
        draw.text((5, coord[1]), text, (0,0,0), font=font)
        coord = (coord[0], coord[1]+50)
        # carga los símbolos
        dicc_Simbolos = obtener_simbolos( (100,100), reducirTransiciones)
        # muestra los símbolos
        for s in dicc_Simbolos.keys():
          coord_simb = agregar_simbolo(coord, page, s)
          draw.text((coord[0]+200, coord[1]), s, (0,0,0), font=font)
          coord = ( coord[0], coord_simb[1][1]+15)
    # muestra la imagen
    display( page )


def contar_cantidad_casos(path_destino):
  # indica cantidad de casos generados
  casosGenerados = os.listdir(path_destino)
  print("\n")
  print("-- Total de casos PNG disponibles: ", len( [ar for ar in casosGenerados if ar.find("png")>0] ))
  print("-- Total de casos XML disponibles: ", len( [ar for ar in casosGenerados if ar.find("xml")>0] ))
  print("\n")
  return


print("Funciones generales definidas.")

In [ ]:
#@title Funciones Especiales para generar XML

def generar_documento_xml(nombre):
  nombre += ".png"
  xml_annotation = ET.Element('annotation')
  ET.SubElement(xml_annotation, 'folder').text = path_destino
  ET.SubElement(xml_annotation, 'filename').text = nombre
  ET.SubElement(xml_annotation, 'path').text = path_destino + nombre

  xml_source = ET.SubElement(xml_annotation, 'source')
  ET.SubElement(xml_source, 'database').text = 'Unknown'

  xml_size = ET.SubElement(xml_annotation, 'size')
  ET.SubElement(xml_size, 'width').text = str(page_size[0])
  ET.SubElement(xml_size, 'height').text = str(page_size[1])
  ET.SubElement(xml_size, 'depth').text = str(3)

  ET.SubElement(xml_annotation, 'segmented').text = '0'

  return xml_annotation


def agregar_anotacion_simbolo(origen, destino, simbolo):
  global xml_annotation
  simbolo = simbolo.replace(".png", "")

  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = simbolo
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  # revisa coordenadas
  xmin = origen[0]
  ymin = origen[1]
  xmax = destino[0]
  ymax = destino[1]
  if xmin > xmax:
    aux = xmin
    xmin = xmax
    xmax = aux
  if ymin > ymax:
    aux = ymin
    ymin = ymax
    ymax = aux

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)
  return

def agregar_anotacion_temporalidad(ymin, ymax, xmin, xmax):
  global xml_annotation
  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = "temporalidad"
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)
  return


def agregar_anotacion_transicion(origen, destino, simbolo="trasladoDeInformacion"):
  global xml_annotation
  
  # revisa tipo y coordenadas
  if "horizontal" in simbolo or "vertical" in simbolo:
    simbolo = simbolo.replace(".png", "")
    xmin = origen[0] 
    ymin = origen[1]
    xmax = destino[0]
    ymax = destino[1]
    if xmin > xmax:
      aux = xmin
      xmin = xmax
      xmax = aux
    if ymin > ymax:
      aux = ymin
      ymin = ymax
      ymax = aux
    if "horizontal" in simbolo:
        xmin += 1
        xmax -= 1
        ymin -= 20
        ymax += 20
    elif "vertical" in simbolo:
        ymin += 1
        ymax -= 1
        xmin -= 20
        xmax += 20
  else:
    # transición copia
    if origen[1] < destino[1]:
      ymax = destino[1]-5
      ymin = origen[1]-5
    else:
      ymax = origen[1]-5
      ymin = destino[1]-5
    if origen[0] < destino[0]:
      xmax = destino[0]-20
      xmin = origen[0]-20
    else:
      xmax = origen[0]+20
      xmin = destino[0]+20

  object = ET.SubElement(xml_annotation, 'object')
  ET.SubElement(object, 'name').text = simbolo
  ET.SubElement(object, 'pose').text = 'Unspecified'
  ET.SubElement(object, 'truncated').text = '0'
  ET.SubElement(object, 'difficult').text = '0'

  xml_bndbox = ET.SubElement(object, 'bndbox')
  ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
  ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
  ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
  ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)
  return


def agregar_anotacion_area(posicion_x, area_interna):

  if posicion_x > 0:
    global xml_annotation
    name = "separador_area_interna" if area_interna else "separador_area_externa"
    object = ET.SubElement(xml_annotation, 'object')
    ET.SubElement(object, 'name').text = name
    ET.SubElement(object, 'pose').text = 'Unspecified'
    ET.SubElement(object, 'truncated').text = '0'
    ET.SubElement(object, 'difficult').text = '0'

    auxXMin = posicion_x - 10
    if auxXMin < 0:
      auxXMin = 1
    auxXMax = posicion_x + 10
    if auxXMax > page_size[0]:
      auxXMax = page_size[0] - 1

    xml_bndbox = ET.SubElement(object, 'bndbox')
    ET.SubElement(xml_bndbox, 'xmin').text = str(auxXMin)
    ET.SubElement(xml_bndbox, 'ymin').text = str(0)
    ET.SubElement(xml_bndbox, 'xmax').text = str(auxXMax)
    ET.SubElement(xml_bndbox, 'ymax').text = str(page_size[1])
  return

def marcar_documento_error():
  global nombre, xml_annotation
  nombre += + "_E"
  xml_annotation.find("filename").text = nombre + ".png"
  xml_annotation.find("path").text = path_destino + nombre + ".png"
  return

print("Funciones Especiales para generar XML definidas.")

In [ ]:
#@title Funciones para Manejo de Reglas


def elegir_inicio(reglas, externo):
  inicios = [regla for regla in reglas if regla["origen"] == ""]
  inicios = inicios if not externo else [inicio for inicio in inicios if not inicio["exclusivo_interno"]]
  inicios = inicios if posicion[1] == 0 else [inicio for inicio in inicios if not "conector" in inicio["destino"]]
  return seleccionarReglaSimbolo(inicios, externo)


def obtener_reglas_posibles(origen):
  global reglas, areas_generadas
  area_actual = areas_generadas[-1]
  reglas_posibles = [regla for regla in reglas if regla["origen"] == origen]
  reglas_posibles = reglas_posibles if not area_actual["externo"] else [regla for regla in reglas_posibles if not regla["exclusivo_interno"] ]
  reglas_posibles = reglas_posibles if generar_errores else [regla for regla in reglas_posibles if not regla["error"]]
  shuffle(reglas_posibles)
  return reglas_posibles


def elegir_accion(origen):
  global areas_generadas
  area_actual_externa = areas_generadas[-1]["externo"] 
  global posicion
  accion = None
  termino = False
  reglas_posibles = obtener_reglas_posibles(origen)
  
  while not termino:
    regla = seleccionarReglaSimbolo(reglas_posibles, area_actual_externa)
    ##regla = choice(np.array(reglas_posibles)) if reglas_posibles else None
    regla_aplicable, movimientos = puedo_aplicar_reglar(regla) if regla is not None else (False, [])

    if regla is not None and (not regla_aplicable or (posicion[1] == 0 and regla["destino"] == "")):
      reglas_posibles.remove(regla)
    else:
      accion = {
          "regla": regla,
          "movimientos": movimientos
      }
      termino = True
  
  return accion


def seleccionarReglaSimbolo(reglasDisponibles, esSectorExterno=False):
  # si no hay nada para seleccionar, devuelve "None"
  if (reglasDisponibles is None) or (not reglasDisponibles) or (len(reglasDisponibles)==0):
    return None
  # desordena
  shuffle(reglasDisponibles)  
  if forzar_decision and not uso_decision and randint(0,5)<4: 
      # si se quiere forzar una decisión y todavía no se uso 
      # y al azar se decide usarla, trata de elegirlo si corresponde
      for sel in reglasDisponibles:
        if "decision" in sel["destino"]:
          break         
  elif forzar_actualizacion and not uso_actualizacion and randint(0,5)<4: 
      # si se quiere forzar una actualización  todavía no se uso 
      # y al azar se decide usarla, trata de elegirla si corresponde
      for sel in reglasDisponibles:
        nombreSimbolo = sel["destino"]
        if "actualizacion" in nombreSimbolo:
          break      
  elif forzar_consulta_logica and not uso_consulta_logica and randint(0,5)<4: 
      # si se quiere forzar una  consulta y todavía no se uso 
      # y al azar se decide usarla, trata de elegirlasi corresponde
      for sel in reglasDisponibles:
        nombreSimbolo = sel["destino"]
        if "consulta" in nombreSimbolo:
          break             
  else:  
      if usar_config_probabilidad_simbolos:   
        # selecciona considerando las probabilidades de la configuración 
        inicios_weights = []
        if esSectorExterno:
          config_posProbab = config_posProbabUsarSectorExterno  
        else:
          config_posProbab = config_posProbabUsarSectorInterno  
        for r in reglasDisponibles:
          nombreSimbolo = r["destino"]
          if nombreSimbolo in config_simbolos.keys():
            inicios_weights.append( config_simbolos[nombreSimbolo][config_posProbab] )
          else:
            inicios_weights.append( config_Probab_Defecto )
        sel = choices(np.array(reglasDisponibles), weights=inicios_weights)[0]
      else:
          # selecciona totalmente al azar 
          sel = choice(np.array(reglasDisponibles))    
  return sel


def posiciones_usadas(posicion_transicion, posicion_simbolo):
  global areas_generadas

  area_actual = areas_generadas[-1]
  columna_otra_area = False
  posicion_usada = any(tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_transicion)) or tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  if not posicion_usada:
    posicion_usada = any(tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_transicion)) or tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  for area in areas_generadas:
    if area != area_actual:
      columna_otra_area = any(int(simbolo["posicion"][0]) == int(posicion_transicion[0]) or int(simbolo["posicion"][0]) == int(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if not columna_otra_area:
        columna_otra_area = any(round(simbolo["posicion"][0]) == round(posicion_transicion[0]) or round(simbolo["posicion"][0]) == round(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if columna_otra_area:
        puede_aplicar = False
        break

  return posicion_usada, columna_otra_area


def validar_posicion_usada(posicion_simbolo):
  global areas_generadas

  area_actual = areas_generadas[-1]
  columna_otra_area = False
  posicion_usada = any(tuple(map(int, simbolo["posicion"])) == tuple(map(int, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  if not posicion_usada:
    posicion_usada = any(tuple(map(round, simbolo["posicion"])) == tuple(map(round, posicion_simbolo)) for simbolo in area_actual["simbolos"])

  for area in areas_generadas:
    if area != area_actual:
      columna_otra_area = any(int(simbolo["posicion"][0]) == int(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if not columna_otra_area:
        columna_otra_area = any(round(simbolo["posicion"][0]) == round(posicion_simbolo[0]) for simbolo in area["simbolos"])

      if columna_otra_area:
        puede_aplicar = False
        break

  return posicion_usada, columna_otra_area


def puede_aplicar_decision(movimientos):
  global posicion, posicion_limite, areas_generadas

  caminos_decision = []
  posible_posicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
  posible_posicion = tuple(np.asarray(posible_posicion) + np.asarray(movimientos[1]))

  movimiento = (0, 1)
  posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
  posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
  posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

  puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[1] <= posicion_limite[1]
  if not puede_usar:
    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if puede_usar:    
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:        
        caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )
  else:    
    caminos_decision.append(
        {
            "transicion": True,
            "simbolo": "transicion_dummy",
            "posicion": posicion_transicion
        }
    )
    caminos_decision.append(
        {
            "transicion": False,
            "simbolo": "simbolo_dummy",
            "posicion": posicion_simbolo
        }
    )

    movimiento = (choice([-1, 1]), 0)
    posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    if not puede_usar:
      movimiento = (-1, 0) if movimiento == (1, 0) else (1, 0)
      posicion_transicion = tuple(np.asarray(posible_posicion) + np.asarray(movimiento))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimiento))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
      puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

      if puede_usar:
        caminos_decision.append(
            {
                "transicion": True,
                "simbolo": "transicion_dummy",
                "posicion": posicion_transicion
            }
        )
        caminos_decision.append(
            {
                "transicion": False,
                "simbolo": "simbolo_dummy",
                "posicion": posicion_simbolo
            }
        )

    else:
      caminos_decision.append(
          {
              "transicion": True,
              "simbolo": "transicion_dummy",
              "posicion": posicion_transicion
          }
      )
      caminos_decision.append(
          {
              "transicion": False,
              "simbolo": "simbolo_dummy",
              "posicion": posicion_simbolo
          }
      )

  if puede_usar:
    areas_generadas[-1]["simbolos"].extend(caminos_decision)

  return puede_usar


def puedo_aplicar_reglar(regla):
  global posicion, posicion_limite, areas_generadas

  simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["posicion"] == posicion]

  if len(simbolos) > 0 and "copia" in simbolos[0]:
    if regla["cambio_area"]:
      return False, []

    regla["copia"] = simbolos[0]["copia"]

    if "movimiento" in simbolos[0]:
      regla["movimiento"] = simbolos[0]["movimiento"]
  else:
    regla["copia"] = 0

  movimientos = []
  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    movimiento = (1, 0)
    movimientos.append(movimiento)
    movimiento = (1, 0) if "actualizacion" in regla["transicion"] else (0, -1)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))

    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0

    return puede_aplicar, movimientos

  if "decision" in regla["origen"]:
    area_actual = areas_generadas[-1]
    simbolos_decision = [simbolo for simbolo in area_actual["simbolos"] if "dummy" in simbolo["simbolo"]]
    movimiento = tuple(np.asarray(simbolos_decision[1]["posicion"]) - np.asarray(simbolos_decision[0]["posicion"]))
    if ("vertical" in regla["transicion"] and movimiento[1] == 1) or ("horizontal" in regla["transicion"] and abs(movimiento[0]) == 1):
        movimientos.append(movimiento)
        movimientos.append(movimiento)

        return True, movimientos

    return False, movimientos

  if "vertical" in regla["transicion"]:
    movimiento = (0, 1)
    movimientos.append(movimiento)
    movimientos.append(movimiento)

    posicion_transicion = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
    posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[1]))
    posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)
    puede_aplicar = (not posicion_usada) and posicion_simbolo[1] <= posicion_limite[1]

    if puede_aplicar and not generar_errores and "proceso" in regla["origen"]:
      area_actual = areas_generadas[-1]
      simbolo_anterior = area_actual["simbolos"][-3]["simbolo"] if len(area_actual["simbolos"]) >= 3 else ""
      puede_aplicar = False if "documento" in simbolo_anterior and "documento" in regla["destino"] else True

    if puede_aplicar and ("archivo" in regla["origen"] or "proceso" in regla["origen"]):
      area_actual = areas_generadas[-1]
      puede_aplicar = False if len(area_actual["simbolos"]) >= 3 and (not "temporalidad" in area_actual["simbolos"][-2]["simbolo"]) else True

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    if puede_aplicar and "operacion" in regla["origen"]:
      generar_copias_documento_operacion(regla, movimientos)

    return puede_aplicar, movimientos

  if "horizontal" in regla["transicion"]:
    columna = choice([-1, 1])
    movimiento = (columna, 0)

    columnas_cruzar = 1
    if regla["cambio_area"]:
      area_actual = areas_generadas[-1]
      columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
      columnas_cruzar = int(math.ceil(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0])) + 1
      simbolos_usados = len([simbolo for simbolo in area_actual["simbolos"] if not simbolo["transicion"]])

      #es para evitar la transicion de area cuando hay un solo simbolo
      if simbolos_usados <= 1:
        return False, movimientos

    movimientos.append(movimiento)

    puede_aplicar = False
    posicion_prueba = posicion
    for i in range(columnas_cruzar):
      movimientos.append(movimiento)

      posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
      posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
      posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

      puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
      posicion_prueba = posicion_simbolo

      if not puede_aplicar:
        break

    if not puede_aplicar:
      movimientos = []
      columna = 1 if columna == -1 else -1
      movimiento = (columna, 0)

      columnas_cruzar = 1
      if regla["cambio_area"]:
        area_actual = areas_generadas[-1]
        columnas_usadas = [simbolo["posicion"][0] for simbolo in area_actual["simbolos"] if not simbolo["transicion"]]
        columnas_cruzar = int(math.ceil(posicion[0] - min(columnas_usadas) if columna == -1 else max(columnas_usadas) - posicion[0])) + 1   

      movimientos.append(movimiento)

      posicion_prueba = posicion
      for i in range(columnas_cruzar):
        movimientos.append(movimiento)

        posicion_transicion = tuple(np.asarray(posicion_prueba) + np.asarray(movimientos[-2]))
        posicion_simbolo = tuple(np.asarray(posicion_transicion) + np.asarray(movimientos[-1]))
        posicion_usada, columna_otra_area = posiciones_usadas(posicion_transicion, posicion_simbolo)

        puede_aplicar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
        posicion_prueba = posicion_simbolo

        if not puede_aplicar:
          break

    if puede_aplicar and "decision" in regla["destino"]:
      puede_aplicar = puede_aplicar_decision(movimientos)

    return puede_aplicar, movimientos

  return False, []



def generar_copias_documento_operacion(regla, movimientos):
  global posicion, posicion_limite, areas_generadas
  #simbolo = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["posicion"] == posicion][0]
  #original = simbolo["copia"] == 0
  original = regla["copia"] == 0

  if forzar_copia and not uso_copia:
    # fuerza uso de copias
    copias = randint(2, 3) if original else 1
  else:
    # lo deja al azar
    copias = randint(1, 3) if original else 1
  posicion_origen = tuple(np.asarray(posicion) + np.asarray(movimientos[0]))
  posicion_origen = tuple(np.asarray(posicion_origen) + np.asarray(movimientos[1]))
  copias_generadas = []

  movimiento_anterior = (0, 0)
  copia = 1
  desplazamiento = 1.5

  for i in range(1, copias):
    if movimiento_anterior != (0, 0):
      movimiento = (desplazamiento, 0) if movimiento_anterior == (-desplazamiento, 0) else (-desplazamiento, 0)
    else:
      movimiento = (choice([-desplazamiento, desplazamiento]), 0)

    posicion_simbolo = tuple(np.asarray(posicion_origen) + np.asarray(movimiento))

    posicion_usada, columna_otra_area = validar_posicion_usada(posicion_simbolo)
    puede_usar = not (posicion_usada or columna_otra_area) and posicion_simbolo[0] <= posicion_limite[0] and posicion_simbolo[0] >= 0
    
    if puede_usar:
      copias_generadas.append(
              {
                  "transicion": False,
                  "simbolo": regla["destino"] + "_copia",
                  "copia": copia,
                  "posicion": posicion_simbolo,
                  "movimiento": movimiento
              }
          )
      copia += 1
    
    movimiento_anterior = movimiento
    
  areas_generadas[-1]["simbolos"].extend(copias_generadas)
  

print("Funciones para Manejo de Reglas definidas")

In [ ]:
#@title Funciones de Generación de Símbolos

def registrar_transicion_en_areas_generadas(tipo_transicion, posicion, coord_dibujo, regla=None):
  global areas_generadas
  if coord_dibujo is None:
    coord_dibujo = ((0,0), (0,0))  
  if not( regla is None ) and  "decision" in regla["origen"]:
    # si es una decisión actualiza la transición "dummy"
    simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "transicion_dummy" and simbolo["posicion"] == posicion]
    if len(simbolos)>0:
      simbolo = simbolos[0]
      simbolo["simbolo"] = tipo_transicion
      simbolo["coord_dibujo"] = coord_dibujo
      return True
    else:
      print("!! Error: no se encontró transicion_dummy en posicion: ", posicion, " en ", areas_generadas[-1]["simbolos"])
      # fuerza a buscar cualquier "dummy"
      simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "transicion_dummy" ]
      if len(simbolos)>0:
        simbolo = simbolos[0]
        simbolo["simbolo"] = tipo_transicion
        simbolo["coord_dibujo"] = coord_dibujo
      return False
  else:
    # agrega transición en área
    areas_generadas[-1]["simbolos"].append(
        {
            "transicion": True,
            "simbolo": tipo_transicion,
            "posicion": posicion,
            "coord_dibujo": coord_dibujo
        }
    )
    return True

def registrar_simbolo_en_areas_generadas(nombre_simbolo, posicion, coord_dibujo, regla=None, nroCopia=-1):
  global areas_generadas
  if coord_dibujo is None:
    coord_dibujo = ((0,0), (0,0))
  if not( regla is None ) and regla["cambio_area"]:
    areas_generadas.append(
        {
            "externo": False,
         # El área externa siempre es la primera o ninguna
         # not regla["exclusivo_interno"] and posicion[0] == 0,
            "simbolos": []
        }
    )
  if not( regla is None ) and "decision" in regla["origen"]:
    # si es una decisión actualiza el símbolo "dummy"
    simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "simbolo_dummy" and simbolo["posicion"] == posicion]
    if len(simbolos)>0:
      simbolo = simbolos[0]
      simbolo["simbolo"] = nombre_simbolo
      simbolo["coord_dibujo"] = coord_dibujo
      return True
    else:
      print("!! Error: no se encontró simbolo_dummy en posicion: ", posicion, " en ", areas_generadas[-1]["simbolos"])
      # fuerza a buscar cualquier "dummy"
      simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if simbolo["simbolo"] == "simbolo_dummy" ]
      if len(simbolos)>0:
        simbolo = simbolos[0]
        simbolo["simbolo"] = nombre_simbolo
        simbolo["coord_dibujo"] = coord_dibujo
      return False
  elif nroCopia > 0:
    # actualiza la copia
    simbolos = [simbolo for simbolo in areas_generadas[-1]["simbolos"] if nombre_simbolo in simbolo["simbolo"] and "copia" in simbolo and simbolo["copia"] == nroCopia]
    if len(simbolos)>0:
      simbolo = simbolos[0]
      simbolo["simbolo"] = nombre_simbolo
      simbolo["coord_dibujo"] = coord_dibujo
      return True
    else:
      print("!! Error: no se encontró nro copia ", nroCopia, "de símmbolo ", nombre_simbolo, " en ", areas_generadas[-1]["simbolos"])
      return False
  else:
    # agrega símbolo en área
    areas_generadas[-1]["simbolos"].append(
        {
          "transicion": False,
          "simbolo": nombre_simbolo,
          "posicion": posicion,
          "coord_dibujo": coord_dibujo
        }
    )
    return True


def cal_posOrigen_simbolo(row, col, tamSimbolo, esInicio):    
  coord = (int( (col * box_size[0]) + (0.5*(box_size[0] - tamSimbolo[0])) + tamSimbolo[0]//2 ), int( (row * box_size[1]) + (0.5*(box_size[1] - tamSimbolo[1])) ) )
  if esInicio:
    # ajusta posición para primera columna
    if col == 0:
      coord = (coord[0]+box_size[0]//5, coord[1])
    # si genera titulos, desplaza todo un poco más abajo
    if seAgreganTransiciones and generar_titulos_areas:
      coord = (coord[0], box_size[1]//2 + 10)
    else:
      coord = (coord[0], 10)  
  return coord


def agregar_inicio(inicio, page, posicion, coord_origen_inicio=None, esInicioAreaExterna=False):
  global areas_generadas
  if coord_origen_inicio==None or len(coord_origen_inicio)==0:
    coord_origen_inicio = cal_posOrigen_simbolo(posicion[1], posicion[0], dicc_Simbolos[ inicio["destino"] ][0], True)
  if modo_DEBUG:
    print("\t- agregar_inicio: ", inicio, "\n\t\tposicion: ", posicion, "destino: ",  inicio["destino"], " coord_origen_inicio simbolo:",  coord_origen_inicio)
  coord_dibujo = agregar_simbolo(coord_origen_inicio, page, inicio["destino"])
  if areas_generadas==None or len(areas_generadas)==0:
    areas_generadas = [
      {
          "externo":  esInicioAreaExterna,
          "simbolos": [
              {
                  "transicion": False,
                  "simbolo": inicio["destino"],
                  "posicion": posicion,
                  "coord_dibujo": coord_dibujo
              }
          ]
      }
    ]    
  else:
    areas_generadas[-1]["simbolos"].append(
      {
          "transicion": False,
          "simbolo": inicio["destino"],
          "posicion": posicion,
          "coord_dibujo": coord_dibujo
      }
    )
  return coord_dibujo, inicio["destino"]
  

def agregar_accion(accion, page, posicion, ultima_coord_dibujo):
  # inicializa las variables
  regla = accion["regla"]
  movimientos = accion["movimientos"]
  posicion_original = posicion
   
  # se fija y agrega las copias que correspondan cambiando la columna si lo hace
  agregaCopias, ultima_coord_dibujo =  agregar_transicion_copia(page, ultima_coord_dibujo, posicion, regla)

  # determina nueva posicion 
  # usando la cantidad de movimientos a realizar 
  # ojo que posicion y movimiento tienen (columna, renglon)  
  cantMovs = [0, 0]
  i = 0
  while i < len(movimientos):
    mov = movimientos[i]
    posicion = tuple(np.asarray(posicion) + np.asarray(mov))
    if i <= (len(movimientos)-2):
      cantMovs = cantMovs + np.asarray(mov)
      if i == (len(movimientos)-2):
        posicion_para_transicion = posicion
    i = i + 1
  # determina coordenadas para el dibujo
  # usnado el consolidado de movimientos a realizar  
  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    # si la transición es una consulta o actualizacion hace un leve desplazamiento especial
    if "actualizacion" in regla["transicion"]:
      if seAgreganTransiciones:
        desp = [dicc_Simbolos[ regla["transicion"] ][0][0], 0.2]
        if reducirTransiciones:
          desp = [desp[0] + 29, desp[1]]
        else:
          desp = [desp[0] + 30, desp[1]]
      else:
        desp = [0.5*box_size[0], 0.2]
    else:
      if seAgreganTransiciones:
        if reducirTransiciones:
          desp = [0.5*box_size[0], -0.3*box_size[1]]
        else:
          desp = [box_size[0], -0.3*box_size[1]]
      else:
        desp = [0.6*box_size[0], -0.1*box_size[1]]
    nueva_coord_origen = (ultima_coord_dibujo[1][0] + int(desp[0]), ultima_coord_dibujo[0][1] + int(desp[1]))      
  else:
      if "vertical" in regla["transicion"]:
        # si es tansición vertical
        nueva_coord_origen = (ultima_coord_dibujo[0][0]+(ultima_coord_dibujo[1][0]-ultima_coord_dibujo[0][0])//2, ultima_coord_dibujo[1][1] + int(abs(cantMovs[1])*distancia_entre_simbolos))
      elif  "horizontal" in regla["transicion"]:      
        # si es transición horizontal (cambio de área  o no)
        # refuerza distancia para que no se superponga
        if seAgreganTransiciones:
          dist = distancia_entre_simbolos * 2          
        else:
          dist = distancia_entre_simbolos * 6
        if regla["cambio_area"]:
          distMin = box_size[1] * 1.5
        else:
          distMin = box_size[1]  
        # controla que haya una distancia mínima 
        # para cambios de área                    
        if dist < distMin:
          dist = distMin
        # tiene en cuenta si es a la izquierda o derecha del último símbolo
        if cantMovs[0] < 0:
          nueva_coord_origen = (ultima_coord_dibujo[0][0] + int(cantMovs[0]*dist), ultima_coord_dibujo[0][1])  
        else:
          nueva_coord_origen = (ultima_coord_dibujo[1][0] + int(cantMovs[0]*dist), ultima_coord_dibujo[0][1])    
      else: 
        dist = distancia_entre_simbolos 
        nueva_coord_origen = (ultima_coord_dibujo[1][0] + int(cantMovs[0]*dist), ultima_coord_dibujo[0][1])    
  # determina símbolo a agregar correspondiente
  nroCopia = -1
  if "documento" in regla["origen"] and regla["origen"] == regla["destino"]:
    # es un traspaso de documento por lo que se marca para usar misma referencia
    nroCopia = 99
  # controla que si tiene que poner un símbolo que cierre el diagrama
  destino = regla["destino"]  
  if posicion[1] >= (posicion_limite[1]-1):
    if not "archivo" in regla["destino"] and not "proceso" in regla["destino"] and not "destruccion" in regla["destino"] and not "decision" in regla["destino"]:  
      destino = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]
  # muestra debug
  if modo_DEBUG:
    print("\t- agregar_accion: ", accion, "\n\t\tposicion: ", posicion, " - destino: ", destino, " - nueva_coord_origen simbolo:",  nueva_coord_origen)
  # controla que no se pase del alto o ancho de la imagen
  if (nueva_coord_origen[0] < 0) or (nueva_coord_origen[0] > page.size[0]) \
    or (nueva_coord_origen[1] < 0) or ((nueva_coord_origen[1] + box_size[1]) > page.size[1]):
    # se cancela su uso por ubicar símbolo fuera del dibujo
    if modo_DEBUG:
      print("\t~ agregar_accion: cancelada por intentar ubicar fuera de la imagen: ", nueva_coord_origen, "!!")
    # igual la regista como hecha en el área para no tener problemas
    registrar_simbolo_en_areas_generadas(destino, posicion, None, regla)
    registrar_transicion_en_areas_generadas(regla["transicion"], posicion_para_transicion, None, regla)
    return posicion_original, ultima_coord_dibujo, regla["origen"]
  # agrega dibujo y anotación del símbolo 
  nueva_coord_dibujo = agregar_simbolo(nueva_coord_origen, page, destino, nroCopia)
  # registra el símbolo en el área
  registrar_simbolo_en_areas_generadas(destino, posicion, nueva_coord_dibujo, regla)
  if seAgreganTransiciones:
    # agrega dibujo y anotación de la transición
    coord_transicion = agregar_simbolo_transicion(ultima_coord_dibujo, nueva_coord_dibujo, page, regla["transicion"], regla)
  else:
    coord_transicion = []
  # registra igual la transición como hecha (la usa para controles internos)
  registrar_transicion_en_areas_generadas(regla["transicion"], posicion_para_transicion, coord_transicion, regla)
  if "actualizacion" in regla["transicion"] or "consulta" in regla["transicion"]:
    # si es actualización o consulta, la próxima regla continua del símbolo origen
    posicion = posicion_original
    nueva_coord_dibujo = ultima_coord_dibujo
    nuevo_origen = regla["origen"] 
  else:
    # para otros símbolos, continua desde el destino
    nuevo_origen = regla["destino"] 
  return posicion, nueva_coord_dibujo, nuevo_origen


def agregar_simbolo(coord_Origen, page, simbolo, nroCopia=-1):      
  # obtiene símbolo a agregar (se hace copia por cambios de referencias)
  img_simbolo = copy.deepcopy( dicc_Simbolos[ simbolo ][1] )  
  # determina el resto de las coordenas del símbolo
  coord_Origen = (coord_Origen[0] - img_simbolo.size[0]//2, coord_Origen[1])
  if coord_Origen[0] <= 0:
    coord_Origen = (1, coord_Origen[1])
  coord_Destino = (coord_Origen[0] + img_simbolo.size[0], coord_Origen[1] + img_simbolo.size[1])
  # agrega la referencia en la imagen del símbolo
  if "documento" in simbolo:
    agregar_referencia_documento(img_simbolo, simbolo, nroCopia)
  elif "conector_comienzo" in simbolo:
    agregar_letras_conector_comienzo(img_simbolo)
  elif "conector_reanudacion" in simbolo:
    agregar_letras_conector_reanudacion(img_simbolo)
  elif "operacion" in simbolo or "control" in simbolo:
    agregar_numero_operacion_control(img_simbolo, ("control" in simbolo))
  elif "decision" in simbolo:
    agregar_numero_decision(img_simbolo)
  elif "archivoTransitorio" in simbolo:
    agregar_numero_archivo_transitorio(img_simbolo)  
  elif "archivoDefinitivo" in simbolo:
    agregar_numero_archivo_definitivo(img_simbolo)  
  elif "proceso" in simbolo and not 'procesoNoRelevado' in simbolo:
    agregar_numero_proceso(img_simbolo)
  # agrega el simbolo en la imagen
  page.paste(img_simbolo, coord_Origen)
  # agrega la anotación en XML
  agregar_anotacion_simbolo(coord_Origen, coord_Destino, simbolo)
  return [coord_Origen, coord_Destino]


def agregar_simbolo_transicion(coord_simbolo_origen, coord_simbolo_destino, page, tipoTransicion, regla):   
  if "trasladoDeInformacion_horizontal" in tipoTransicion or "trasladoDeInformacion_vertical" in tipoTransicion:      
      # determina posiciones de acuerdo a tipo de transición
      # y ubicación real de los símbolos
      if "vertical" in tipoTransicion:
          if abs(coord_simbolo_destino[0][1]-coord_simbolo_origen[1][1]) < abs(coord_simbolo_destino[1][1]-coord_simbolo_origen[0][1]):
              coord_origen_y = coord_simbolo_origen[1][1]
              coord_destino_y = coord_simbolo_destino[0][1]
          else:
              coord_origen_y = coord_simbolo_origen[0][1]
              coord_destino_y = coord_simbolo_destino[1][1]
          coord_Origen = ( coord_simbolo_origen[0][0] + (coord_simbolo_origen[1][0]-coord_simbolo_origen[0][0])//2, coord_origen_y )
          coord_Destino = ( coord_simbolo_destino[0][0] + (coord_simbolo_destino[1][0]-coord_simbolo_destino[0][0])//2, coord_destino_y )
      else:
          if abs(coord_simbolo_destino[0][0]-coord_simbolo_origen[1][0]) < abs(coord_simbolo_destino[1][0]-coord_simbolo_origen[0][0]):
            coord_origen_x = coord_simbolo_origen[1][0]
            coord_destino_x = coord_simbolo_destino[0][0]
          else:
            coord_origen_x = coord_simbolo_origen[0][0]
            coord_destino_x = coord_simbolo_destino[1][0]
          coord_Origen = ( coord_origen_x, coord_simbolo_origen[0][1] + (coord_simbolo_origen[1][1]-coord_simbolo_origen[0][1])//2 )
          coord_Destino = ( coord_destino_x, coord_simbolo_origen[0][1] + (coord_simbolo_origen[1][1]-coord_simbolo_origen[0][1])//2 )
      # dibuja la línea
      if reducirTransiciones:
        coord_Origen = (coord_Origen[0], coord_Origen[1]-1)
      draw = ImageDraw.Draw(page)
      draw.line([coord_Origen, coord_Destino], fill=(0, 0, 0), width=transicion_line_width)
      del draw
      # agrega la anotación en XML
      agregar_anotacion_transicion(coord_Origen, coord_Destino, tipoTransicion)
  else:
      # para otras flechas 
      # obtiene símbolo a agregar (se hace copia por cambios de referencias)
      img_simbolo = copy.deepcopy( dicc_Simbolos[ tipoTransicion ][1] )  
      # determina posición y tamaño del símbolo
      despOrigen = (0, 0)
      if 'actualizacion' in tipoTransicion or 'consulta' in tipoTransicion:
        despOrigen = (despOrigen[0]+1, despOrigen[1])
      coord_Origen = ( coord_simbolo_origen[1][0] + 1 + despOrigen[0], coord_simbolo_origen[0][1] + (coord_simbolo_origen[1][1]-coord_simbolo_origen[0][1])//2 + despOrigen[1] )      
      coord_Destino = (coord_Origen[0] + img_simbolo.size[0], coord_Origen[1] + img_simbolo.size[1])
      # agrega el simbolo en la imagen
      page.paste(img_simbolo, coord_Origen)
      # agrega la anotación en XML
      agregar_anotacion_simbolo(coord_Origen, coord_Destino, tipoTransicion)
  # devuelve la posición
  return [coord_Origen, coord_Destino]



def agregar_transicion_copia(page, ultima_coord_dibujo, posicion, regla):
  global uso_copia

  # determina si es una copia de documento a agregar
  if regla["copia"] > 0 and "movimiento" in regla:
    
    # determina nueva posicion 
    posicion = tuple(np.asarray(posicion) + np.asarray(regla["movimiento"]))

    # determina nueva coordenadas de la copia
    cantMovs = regla["movimiento"][0]
    if seAgreganTransiciones:
      dist = distancia_entre_simbolos       
    else:
      dist = distancia_entre_simbolos * 3
    distMin = box_size[0]//2
    if dist < distMin:
      dist = distMin
    if cantMovs < 0:
      nueva_coord_origen = (ultima_coord_dibujo[0][0] + int(cantMovs*dist), ultima_coord_dibujo[0][1])  
    else:
      nueva_coord_origen = (ultima_coord_dibujo[1][0] + int(cantMovs*dist), ultima_coord_dibujo[0][1])      
    # agrega símbolo indicando id de copia
    nueva_coord_dibujo = agregar_simbolo(nueva_coord_origen, page, regla["origen"], regla["copia"])
    registrar_simbolo_en_areas_generadas(regla["origen"], posicion, nueva_coord_dibujo, regla, regla["copia"])

    if modo_DEBUG:
      print("\t-agregar_transicion_copia: regla:", regla, "\n\tultima_coord_dibujo:", ultima_coord_dibujo, "- nueva_coord_dibujo:", nueva_coord_dibujo)

    if seAgreganTransiciones:
      draw = ImageDraw.Draw(page)
      # determian coordenadas
      punto_origen = (nueva_coord_dibujo[0][0]+(nueva_coord_dibujo[1][0]-nueva_coord_dibujo[0][0])//2, nueva_coord_dibujo[0][1])
      punto_destino = (ultima_coord_dibujo[0][0]+(ultima_coord_dibujo[1][0]-ultima_coord_dibujo[0][0])//2, int(nueva_coord_dibujo[0][1]-distancia_entre_simbolos//2))
      punto_intermedio = (punto_origen[0], punto_destino[1])
      # dibuja la parte vertical
      # dibuja la parte horizontal
      draw.line([punto_origen, punto_intermedio, punto_destino], fill=(0, 0, 0), width=transicion_line_width)
      agregar_anotacion_transicion(punto_origen, punto_destino)  

      if modo_DEBUG:
        print("\t\tTransicion de ", punto_origen, "a", punto_destino, "por", punto_intermedio)
    
    # marca como hecho
    uso_copia = True
    del regla["movimiento"]

    return True, nueva_coord_dibujo
  else:
    return False, ultima_coord_dibujo


def dibujar_lineas_areas(areas, page):
  if not seAgreganTransiciones:
    # si no se agregan transiciones, no se generan las líneas de áreas tampoco
    return
  draw = ImageDraw.Draw(page)
  if generar_recuadro_exterior:
    # genera líneas de rectángulo exterior del cursograma
    iniX = 0
    limX = page.size[0]-areas_line_width
    iniY = 0
    limY = page.size[1]-areas_line_width
    draw.line([(iniX, iniY), (limX, iniY)], fill=(0, 0, 0), width=areas_line_width)
    draw.line([(limX, iniY), (limX, limY)], fill=(0, 0, 0), width=areas_line_width)
    draw.line([(iniX,  limY), (limX,  limY)], fill=(0, 0, 0), width=areas_line_width)
    draw.line([(iniX, iniY), (iniX,  limY)], fill=(0, 0, 0), width=areas_line_width)
  if generar_titulos_areas:
    # genera línea para títulos de áreas
    posicion_y_linea_titulo = int(0.5 * box_size[1])
    draw.line([(0, posicion_y_linea_titulo), (page.size[0], posicion_y_linea_titulo)], fill=(0, 0, 0), width=areas_line_width)
    # inicializa font para usar
    font = ImageFont.truetype(font=path_fuente, size=18)
    posicion_y_linea_titulo = 10
  # determina líneas de áreas 
  distArea = distancia_entre_simbolos//2
  if distArea < 20:
    # fuerza una distancia mínima para que se vea bien
    distArea = 20
  posicion_lineas_generar = []
  for area in areas:    
    es_area_externa = area["externo"]
    coord_columna_max = -1
    # determina la posicion de las líneas
    for simbolo in area["simbolos"]:
      if "coord_dibujo" in simbolo and not simbolo["transicion"]:
        # no es transición
        if coord_columna_max < simbolo["coord_dibujo"][1][0]:
          coord_columna_max = simbolo["coord_dibujo"][1][0] 
    # registra posición de las líneas (si están definidas)
    coord_columna_max = int(coord_columna_max + distArea) 
    if coord_columna_max > 0 and coord_columna_max < page.size[0] and ([es_area_externa, coord_columna_max] not in posicion_lineas_generar):
      posicion_lineas_generar.append( [es_area_externa, coord_columna_max] )
  # dibuja las líneas de separación del ára externa e internas
  posicion_lineas_generar.sort(key=lambda x:x[1])
  pos_linea_anterior = 1
  posicion_linea = pos_linea_anterior
  for pos in posicion_lineas_generar:
    es_area_externa = pos[0]
    posicion_linea = pos[1]    
    # si no es una línea repetida y es externa o hay diferencia con la anterior
    if (posicion_linea != pos_linea_anterior) and (es_area_externa or ((posicion_linea - pos_linea_anterior) > box_size[1]//2)):
      # dibuja la línea
      draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=areas_line_width)
      if es_area_externa:
        # dibuja marcas de la línea de área externa
        max_white_lines = int(page.size[1]/white_line_size)
        draw.line([(posicion_linea, 0), (posicion_linea, page.size[1])], fill=(0, 0, 0), width=areas_line_width)      
        for i in range(max_white_lines):
          if i % 2 != 0:
            draw.line([(posicion_linea, i*white_line_size), (posicion_linea, i*white_line_size + white_line_size)], fill=(255, 255, 255), width=areas_line_width)      
      # registra la anotación      
      agregar_anotacion_area(posicion_linea, not es_area_externa)
      # agrega título de área
      if generar_titulos_areas:
        if es_area_externa:
          text = devolverReferenciaSimbolo("areaExterna")   
        else:
          text = devolverReferenciaSimbolo("areaInterna")              
        anchoTexto = font.getsize(text)[0]
        posX = pos_linea_anterior + (posicion_linea-pos_linea_anterior)//2 - anchoTexto//2      
        if posX < pos_linea_anterior or (posX+anchoTexto)>posicion_linea:
          posX = pos_linea_anterior + 1
        draw.text((posX, posicion_y_linea_titulo), text, (0,0,0), font=font) 
      # registra la posición de la línea anterior
      pos_linea_anterior = posicion_linea
  # agrega descripción última área (si corresponde y entra)
  if len(posicion_lineas_generar) > 1 and generar_titulos_areas:
    text = "OBSERVACIONES:"
    anchoTexto = font.getsize(text)[0]
    posX = posicion_linea + 11
    if (posX + anchoTexto + 10) < page.size[0]:
      draw.text((posX, posicion_y_linea_titulo), text, (0,0,0), font=font) 
    else:
      text = "OBS:"
      anchoTexto = font.getsize(text)[0]
      if (posX + anchoTexto + 10) < page.size[0]:
        draw.text((posX, posicion_y_linea_titulo), text, (0,0,0), font=font) 
  del draw
  return


def agregar_temporalidad(page, posicion_maxima, ultima_coord_dibujo):
  global areas_generadas, reglas  
  distEspecial_antes_xml = 40
  distEspecial_texto = distEspecial_antes_xml - 10
  distEspecial_despues_xml = 10
  if seAgreganTransiciones:
    dist_antes = int(distancia_entre_simbolos//2 + distEspecial_antes_xml)
    dist_despues = int(distancia_entre_simbolos//2)
  else:
    dist_antes = int(distancia_entre_simbolos + distEspecial_antes_xml * 2)
    dist_despues = int(distancia_entre_simbolos + distEspecial_despues_xml)
  # determina posicion para temporalidad
  posicion = tuple(np.asarray(posicion_maxima) + np.asarray((0, 1))) 
  posicion_y_linea = int( ultima_coord_dibujo[1][1] + dist_antes )
  # dibuja temporalidad
  draw = ImageDraw.Draw(page)
  draw.line([(0, posicion_y_linea), (page.size[0], posicion_y_linea)], fill=(0, 0, 0), width=temporalidad_line_width)
  max_white_lines = int(page.size[0]/white_line_size)
  for i in range(max_white_lines):
    if i % 2 != 0:
      draw.line([(i*white_line_size, posicion_y_linea), (i*white_line_size + white_line_size, posicion_y_linea)], fill=(255, 255, 255), width=temporalidad_line_width)
  # agrega etiqueta
  usar_label = True if not generar_errores else choice([True, False])
  if usar_label:
    text = devolverReferenciaSimbolo("temporalidad.png")
    #if seAgreganTransiciones and not reducirTransiciones:
    font = ImageFont.truetype(font=path_fuente, size=18)
    draw.text((5, posicion_y_linea-distEspecial_texto), text,(0,0,0), font=font) # siempre va a la izquierda
  else:
    marcar_documento_error()
  # agrega en XML
  agregar_anotacion_temporalidad(posicion_y_linea-distEspecial_antes_xml, posicion_y_linea+distEspecial_despues_xml, 0, page.size[0])
  if modo_DEBUG:
    print("\t- agregar_temporalidad:  posicion_y_linea:",  posicion_y_linea)
  registrar_simbolo_en_areas_generadas("temporalidad", (0, int((0.5 + posicion[1]))), [[0,posicion_y_linea],[page.size[0],posicion_y_linea]])
  # agrega primer símbolo luego
  posicion = tuple(np.asarray(posicion) + np.asarray((0, 1)))
  inicio = elegir_inicio(reglas, areas_generadas[-1]["externo"])
  nCoord_inicio = (int(ultima_coord_dibujo[0][0]+(ultima_coord_dibujo[1][0]-ultima_coord_dibujo[0][0])//2), posicion_y_linea+dist_despues)
  ultima_coord_dibujo, origen = agregar_inicio(inicio, page, posicion, nCoord_inicio)
  # devuelve valores
  del draw
  return ultima_coord_dibujo, origen, posicion

print("Funciones de Generación de Símbolos definidas.")

In [ ]:
#@title Funciones para Referencias de Símbolos 

referencias_valores_numeros_romanos = ["I", "II", "III", "IV", "V", "VI", "VII", "IX", "X"]

aux_anterior_referencia_documento = None

def devolverReferenciaSimbolo(nombreSimbolo):
  global diccReferenciasSecuenciales

  # si se deben generar simbolos sin referencias
  if generar_simbolos_sin_referencias:
    return "  "

  # determina tipo de referencia del símbolo
  if nombreSimbolo in config_simbolos.keys():
    tipoRef = config_simbolos[nombreSimbolo][config_posTipoReferencia]
  else:
    tipoRef = '-'

  # determina la fuente de información para la referencia
  fuenteRef = ''
  if tipoRef == 'A':
    # Número Arábigo (le saca el '0' inicial para que arranque en '1')
    fuenteRef = string.digits[1:] 
  elif tipoRef == 'R':
    # Número Romano
    fuenteRef = referencias_valores_numeros_romanos
  elif tipoRef == 'L':
    # Letra
    fuenteRef = string.ascii_uppercase
  elif tipoRef == 'D':
    # Letras de tipo Documentos
    # (directamente la devuelve porque no tiene secuencia)
    if usar_config_valores_tipo_referencias and tipoRef in config_referencias.keys():
      return choice(config_referencias[tipoRef]).upper()
    else:
      return ''.join(sample(string.ascii_uppercase, randint(1, 2)))
  elif tipoRef == 'T':
    # Textos de Temporalidad
    # (directamente la devuelve porque no tiene secuencia)
    if usar_config_valores_tipo_referencias and tipoRef in config_referencias.keys():
      return choice(config_referencias[tipoRef])
    else:      
      return ''.join(sample(string.ascii_uppercase, randint(4, 10)))
  elif tipoRef == 'AI' or tipoRef == 'AE':
    # Textos de Títulos de Áreas Internas o Externas
    # (controla que no este usada y directamente la devuelve porque no tiene secuencia)
    if usar_config_valores_tipo_referencias and tipoRef in config_referencias.keys():
      refDup = True
      cant = 0
      sel = "-"
      while refDup and cant<10:
        sel = choice(config_referencias[tipoRef]).upper()
        auxIdRef = "REF_" + tipoRef + "_USADA_" + sel
        if auxIdRef not in diccReferenciasSecuenciales:
          diccReferenciasSecuenciales[auxIdRef] = 1
          refDup = False
        else:
          cant = cant + 1
      return sel
    else:      
      return ''.join(sample(string.ascii_uppercase, randint(4, 10)))
  else:
    # No Aplica o "desconocido"
    # (directamente la devuelve porque no tiene secuencia)
    return ""

  # determina tipo de secuencia de referencia a usar
  if usar_referencias_generadas == "AZAR":
    # devuelve un elemento al azar 
    return choice( fuenteRef ) 
  else:
    # obtiene siguiente elemento a usar por tipo de referencia
    if usar_referencias_generadas == "SECUENCIAL_TIPO_REFERENCIA":
      auxIdRef = "REF_TREF_" + tipoRef
    elif usar_referencias_generadas == "SECUENCIAL_SIMBOLO":
      auxIdRef = "REF_S_" + nombreSimbolo
    else:
      return ""   
    if auxIdRef in diccReferenciasSecuenciales:
      auxPos =  diccReferenciasSecuenciales[auxIdRef] + 1  
    else:
      diccReferenciasSecuenciales[auxIdRef] = 0
      auxPos = 0
    # determina al azar si usa el incremento o repite uno ya usado (poca probabilidad)
    if randint(0, 100) > 80:
      # repite
      auxPos = randint(0, auxPos)    
    # controla que no se pase (sino se resetea)
    if auxPos > len(fuenteRef):
      auxPos = 0
    # actualiza el diccionario y devuelve el valor que corresponde
    if diccReferenciasSecuenciales[auxIdRef] < auxPos:
      diccReferenciasSecuenciales[auxIdRef] = auxPos
    # devuelve la referencia que corresponda
    return fuenteRef[ auxPos ]

def agregar_referencia_documento(img_simbolo, nombreSimbolo, nroCopia=-1):   
  global diccReferenciasSecuenciales
  global aux_anterior_referencia_documento
  
  # si se deben generar simbolos sin referencias
  if generar_simbolos_sin_referencias:
    tDoc = "   "
    numero_documento = -1
  else:
    if nroCopia >= 0  and aux_anterior_referencia_documento!=None and len(aux_anterior_referencia_documento)==2:
      # usa el tipo y nro rdel documento anterior
      tDoc = aux_anterior_referencia_documento[0]
      numero_documento = aux_anterior_referencia_documento[1] 
      if nroCopia < 10:
        numero_documento = numero_documento + nroCopia   
        # incrementa el nuevo número como usado
        auxIdRef = "REF_DOC_" + nombreSimbolo + "_" + tDoc  
        if numero_documento<9:
          if auxIdRef in diccReferenciasSecuenciales and diccReferenciasSecuenciales[auxIdRef]<numero_documento:   
            diccReferenciasSecuenciales[auxIdRef] = numero_documento        
    else:
      # resetea documento anterior
      aux_anterior_referencia_documento = None
      # lista auxiliar de tipos de documentos ya creados
      auxRefDoc = []
      for k in diccReferenciasSecuenciales.keys():
        if "REF_DOC_" in k:
          p = k.find(".png_")
          if p < 0:
            p = 8
          else:
            p = p + 5
          auxRefDoc.append( k[p:] )

      # determina el tipo de documento al azar 
      # si reutiliza un documento ya usado o uno nuevo
      if len(auxRefDoc)>0 and randint(0, 100) > 80:  
        tDoc = choice( auxRefDoc )
      else:
        tDoc = devolverReferenciaSimbolo(nombreSimbolo)

      if usar_referencias_generadas == "AZAR":
        # determina número de referencia al azar
        numero_documento = randint(-1, 9)
      else:
        # determina número de referencia secuencial
        auxIdRef = "REF_DOC_" + nombreSimbolo + "_" + tDoc
        if auxIdRef in diccReferenciasSecuenciales:
          # utiliza último generado
          numero_documento =  diccReferenciasSecuenciales[auxIdRef]
          if numero_documento >= 0:
            # determina al azar si usa el incremento o repite uno ya usado (poca probabilidad)
            if randint(0, 100) > 80:
              # repite
              numero_documento = randint(0, numero_documento)   
            else:
              # incrementa el ultimo utilizado
              numero_documento = numero_documento + 1
              if numero_documento < 9:
                diccReferenciasSecuenciales[auxIdRef] = numero_documento
        else:      
          # determina al azar si usa nro de documento o no ("-1")
          if randint(0, 100) <= 90:  
            numero_documento = 0
          else:
            numero_documento = -1      
          diccReferenciasSecuenciales[auxIdRef] = numero_documento
    
      aux_anterior_referencia_documento = [tDoc, numero_documento]

  # agrega tipo documento en la imagen
  agregar_letras_documento(img_simbolo, tDoc)

  # agrega número documento en la imagen
  agregar_numero_documento(img_simbolo, numero_documento)


def agregar_letras_documento(img_simbolo, text):
  displacement = 0 if len(text) == 1 else 14  
  blank_size = (40, 40) if len(text) == 1 else (47, 40)
  blank_position = (24 - displacement, 30)
  tamLetra = 25 if len(text) >= 3 else 28
  if reducirTransiciones:
    blank_position = (blank_position[0]-4, blank_position[1]-5)
    blank_size = (blank_size[0]-5, blank_size[1]-5)
    tamLetra = tamLetra - 2
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=tamLetra)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_documento(img_simbolo, numero):
  blank_size = (17, 23)
  if reducirTransiciones:  
    blank_position = (39, 63)
  else:
    blank_position = (45, 70)
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=20)
  if numero >= 0:
    text =  str(numero)
  else:
    text = "  " 
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)
  
def agregar_letras_conector_comienzo(img_simbolo):
  blank_size = (35, 35)
  if reducirTransiciones:  
    blank_position = (25, 12)
  else:
    blank_position = (37, 22)
  text = devolverReferenciaSimbolo("conector_comienzoCorte.png")
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_letras_conector_reanudacion(img_simbolo):
  blank_size = (35, 35)
  if reducirTransiciones:  
    blank_position = (25, 28)
  else:
    blank_position = (37, 40) 
  text = devolverReferenciaSimbolo("conector_reanudacionCorte.png")  
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_proceso(img_simbolo):
  blank_size = (30, 30)
  blank_position = (40, 5)
  if reducirTransiciones:
    blank_position = (blank_position[0]-3, blank_position[1])
  text = devolverReferenciaSimbolo("proceso.png")  
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_operacion_control(img_simbolo, esControl):
  if esControl:
    text = devolverReferenciaSimbolo("control.png")  
    if reducirTransiciones:  
      blank_size = (30, 35)
      blank_position = (20, 10)    
    else:
      blank_size = (40, 40)
      blank_position = (40, 30)
  else:
    text = devolverReferenciaSimbolo("operacion.png")  
    if reducirTransiciones:  
      blank_size = (29, 30)
      blank_position = (20, 9)    
    else:
      blank_size = (40, 35)
      blank_position = (40, 30)
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=28)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_decision(img_simbolo):
  if reducirTransiciones:  
    blank_size = (30, 22)  
    number_size = (20, 27)
    number_position = (20, 15)
    blank_position = (15, 17)
    tamanioFont = 25
  else:
    blank_size = (35, 33)  
    number_size = (35, 35)
    number_position = (40, 30)
    blank_position = (26, 38)
    tamanioFont = 27
  text = devolverReferenciaSimbolo("decision.png")  
  blank = Image.new("RGB", blank_size, "white")
  number = Image.new("RGB", number_size, "white")
  draw = ImageDraw.Draw(number)
  font = ImageFont.truetype(font=path_fuente, size=tamanioFont)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)
  img_simbolo.paste(number, number_position)

def agregar_numero_archivo_transitorio(img_simbolo):
  text = devolverReferenciaSimbolo("archivoTransitorio.png")  
  if len(text) == 1:
    blank_size = (25, 27)
    blank_position = (40, 8)
  elif len(text) == 2:
    blank_size = (28, 27)
    blank_position = (35, 8)
  elif len(text) >= 3:
    blank_size = (38, 27)
    blank_position = (31, 8)
  tamanioFont = 25
  if reducirTransiciones:
    blank_position = (blank_position[0]-8, blank_position[1]-4)
    blank_size = (blank_size[0]-2, blank_size[1]-3)    
    tamanioFont = 20
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=tamanioFont)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)

def agregar_numero_archivo_definitivo(img_simbolo):
  text = devolverReferenciaSimbolo("archivoDefinitivo.png")  
  if len(text) == 1:
    blank_size = (25, 27)
    blank_position = (40, 32)
  elif len(text) == 2:
    blank_size = (28, 27)
    blank_position = (35, 32)
  elif len(text) >= 3:
    blank_size = (38, 27)
    blank_position = (31, 32)
    tamanioFont = 25
  if reducirTransiciones:
    blank_position = (blank_position[0]-8, blank_position[1]-7)    
    blank_size = (blank_size[0]-3, blank_size[1])    
    tamanioFont = 20
  blank = Image.new("RGB", blank_size, "white")
  draw = ImageDraw.Draw(blank)
  font = ImageFont.truetype(font=path_fuente, size=25)
  draw.text((0, 0), text, (0,0,0), font=font)
  img_simbolo.paste(blank, blank_position)


print("Funciones Especiales para Referencias de Símbolos definidas.") 

### función para probar como quedan las referencias en los símbolos
##debug_probarSimbolos()

5) Generar los Casos de Cursograma con reglas:

In [ ]:
#@title Llevar a cabo la Generación de los Casos

# inicializar carpeta destino
inicializar_carpeta_destino(path_destino, forzar_drive_borrado)

# inicializar variables globales
reglas = cargar_reglas()

print("** Comienza Generación **")

if forzar_generar_combinaciones_parametros:
  # indica la lista de casos a generar
  # (las letras indica el tipo de configuración - ver abajo)
  lista_config_tipos_casos = [ 'SARV', 'SARA', 'SARC', 'SARS', \
                              'SPRV', 'SPRA', 'SPRC', 'SPRS', \
                              'SFdRV','SFcRV','SFeRV','SFaRV','SFoRV', \
                              'SFdRS','SFcRS','SFeRS','SFaRS','SFoRS', \
                              'SFdRA','SFcRA','SFeRA','SFaRA','SFoRA' ]
  # determina cantida de casos
  cantidad_casos = cantidad_casos_generar_total // len(lista_config_tipos_casos) 
  if cantidad_casos > 999:
    cantidad_casos = 999
  elif cantidad_casos <= 0:
    cantidad_casos = 1    
  print(" ++ se detecta activación de [forzar_generar_combinaciones_parametros]:  ")
  print("    cantidad de casos a generar por tipo: ", cantidad_casos, "+++\n")
else:
  lista_config_tipos_casos = [ '-' ]
  cantidad_casos = cantidad_casos_generar_total


# comienza a ciclar
for tipo_caso in lista_config_tipos_casos:
  # si determina los parámetros para cada tipo de combinación
  if tipo_caso == '-':
    prefijoNombreCaso = "curso_" + prefijo_inicial_caso_generado
  else:
    prefijoNombreCaso, tipo_caso_genSimbolos, tipo_caso_genReferencias = determinarConfigTipoCaso(tipo_caso)
    prefijoNombreCaso = "c_" + prefijo_inicial_caso_generado + "_" + prefijoNombreCaso

  # lleva a cabo la generación de casos 
  # considerando los parámetros
  i = valor_inicial_caso_generado
  cantidad_casos = cantidad_casos + valor_inicial_caso_generado - 1
  while i <= cantidad_casos:

    # determina configuración especial del caso
    generar_recuadro_exterior, generar_titulos_areas = determinaOtraConfig()    
    sufijoNombreCaso, page_size, seAgreganTransiciones, reducirTransiciones, \
      distancia_entre_simbolos, box_size, posicion_limite, \
      areas_line_width, temporalidad_line_width, transicion_line_width, white_line_size \
      = determinaConfigTamanios(generar_titulos_areas)    
    dicc_Simbolos = obtener_simbolos( (100,100), reducirTransiciones)

    # determina nombre del caso
    nombre = prefijoNombreCaso + "_" + ("00000" + str(i))[-5:] + sufijoNombreCaso
    print("\n+ Generando #", i, " de ", cantidad_casos, " - ", nombre)
    if modo_DEBUG:
      print("\t--Config Tamaños: seAgreganTransiciones: ", seAgreganTransiciones, "reducirTransiciones: ", reducirTransiciones, "\n\tdistancia_entre_simbolos: ", distancia_entre_simbolos, "page_size: ", page_size)

    # inicializa imagen y XML del caso
    page, xml_annotation = crear_cursograma(nombre, page_size)
    # inicializa variables auxiliares
    uso_decision = False
    uso_copia = False
    uso_area_externa = False
    uso_actualizacion = False
    uso_consulta_logica = False
    # inicializa diccionario para referencias
    diccReferenciasSecuenciales = {}
    # comienza la generación
    ultima_coord_dibujo = (0, 0)
    areas_generadas = None
    esInicioAreaExterna = False
    if forzar_area_externa:
      # si se fuerza area externa, ya la indica
      posicion = (0, 0)
      esInicioAreaExterna = True
      uso_area_externa = True
    else:
      posicion = (randint(0, posicion_limite[0]), 0)
      if posicion[0] == 0:
        #determina al azar si va a ser area externa 
        if randint(0, 10) > 5:
          esInicioAreaExterna = True
      uso_area_externa = esInicioAreaExterna    
    # selecciona símbolo de inicio
    inicio = elegir_inicio(reglas, esInicioAreaExterna)
    # genera símbolo de inicio
    ultima_coord_dibujo, origen = agregar_inicio(inicio, page, posicion, None, esInicioAreaExterna)

    # continua la generación
    decisiones = []
    y_maximo = 0
    x_maximo = 0
    termino = False
    while not termino:
      
      # elige regla a aplicar
      accion = elegir_accion(origen)

      if not(accion is None) and not( accion["regla"] is None) and (accion["regla"]["destino"] != ""):
        # hay regla para aplicar

        # aplica la regla, agregando los símbolos que corresponden
        posicion, ultima_coord_dibujo, origen = agregar_accion(accion, page, posicion, ultima_coord_dibujo)
        
        # determina si símbolos usados
        if accion["regla"]["error"]:
          marcar_documento_error()        
        if "actualizacion" in accion["regla"]["transicion"]:
          uso_actualizacion = True
        if "consulta" in accion["regla"]["transicion"]:
          uso_consulta_logica = True
        if "decision" in origen:
          uso_decision = True

        # si ya está en una posición cercana al límite
        if posicion[1] >= (posicion_limite[1]-1): 
          if not "archivo" in accion["regla"]["destino"] and not "proceso" in accion["regla"]["destino"] and not "destruccion" in accion["regla"]["destino"]:
            origen = [item for item in reglas if item["destino"] == "" and "conector" in item["origen"]][0]["origen"]
        
      else:
        # como no hay reglas para aplicar
        # se fija si hay decisiones o copias incompletas
        areas_con_decisiones = [area for area in areas_generadas if any([simbolo for simbolo in area["simbolos"] if "simbolo_dummy" in simbolo["simbolo"]])]
        areas_con_copias = [area for area in areas_generadas if any([simbolo for simbolo in area["simbolos"] if "_copia" in simbolo["simbolo"]])]

        if len(areas_con_decisiones) > 0:
          # detecta decisiones incompletas
          area_con_decision = areas_con_decisiones[0]
          simbolos_completar_decision = [simbolo for simbolo in area_con_decision["simbolos"] if "_dummy" in simbolo["simbolo"]]
          if modo_DEBUG:
            print("\t-Areas_con_decisiones:  simbolos_completar_decision: ", simbolos_completar_decision)
          movimiento_realizado = np.asarray(simbolos_completar_decision[1]["posicion"]) - np.asarray(simbolos_completar_decision[0]["posicion"])            
          # fuerza a procesar la última decisión para completarla
          simbolo_decision = [simbolo for simbolo in area_con_decision["simbolos"] if "decision" in simbolo["simbolo"]][0]
          if simbolo_decision is None:
            print(" !!Error: no se pudo encontrar la decisión en: ", area_con_decision, "\n\t simbolo_decision:", simbolo_decision)
            print(" Caso descartado!")
            break
          origen = simbolo_decision["simbolo"]                     
          posicion = simbolo_decision["posicion"]                    
          ultima_coord_dibujo = tuple(simbolo_decision["coord_dibujo"])
          if modo_DEBUG:
            print("\t\t Decision: nueva posicion: ", posicion, "movimiento_realizado: ", movimiento_realizado, "\n\tultima_coord_dibujo: ", ultima_coord_dibujo)
          # fuerza que esta área sea la última
          areas_generadas.remove(area_con_decision)
          areas_generadas.append(area_con_decision)

        elif len(areas_con_copias) > 0:
          # detecta copias faltantes
          area_con_copias = areas_con_copias[0]
          simbolo_copia = [simbolo for simbolo in area_con_copias["simbolos"] if "_copia" in simbolo["simbolo"]][0]
          if modo_DEBUG:
            print("\t-Areas_con_copias: ", area_con_copias, "\n\t\t simbolo_copia: ", simbolo_copia)          
          posicion = simbolo_copia["posicion"]
          origen = simbolo_copia["simbolo"].split("_copia")[0]
          simbolo_copia["simbolo"] = origen
          simbolos_original = [simbolo for simbolo in area_con_copias["simbolos"] if origen in simbolo["simbolo"] and (not "copia" in simbolo or simbolo["copia"]==0)]
          if simbolos_original is not None and len(simbolos_original)>0:            
            # como puede haber más de un documento, determina cual usar
            id_doc_original = 0
            while id_doc_original < len(simbolos_original):
                # tiene coordenadas definidas en el dibujo
                if "coord_dibujo" in simbolos_original[id_doc_original]:       
                  # y está en el mismo renglón que la copia
                  if int(simbolos_original[id_doc_original]["posicion"][1]) == int(simbolo_copia["posicion"][1]):
                    # actualiza las coordenadas
                    ultima_coord_dibujo = tuple(simbolos_original[id_doc_original]["coord_dibujo"])
                    if modo_DEBUG:
                      print("\t\t se actualiza ultima_coord_dibujo: ", ultima_coord_dibujo, "con ", simbolos_original[id_doc_original])
                    break
                id_doc_original = id_doc_original + 1             

        else: #if len(areas_con_decisiones) == 0 and len(areas_con_copias) == 0:
          # sino se fija si se puede agregar algo más o finaliza
          y_maximo = -1
          x_maximo = -1
          area_con_maximo = None
          posicion_con_maximo = None
          ultimo_simbolo = None
          for area in areas_generadas:
            for simbolo in area["simbolos"]:
              posicion_area = simbolo["posicion"]
              if posicion_area[1] > y_maximo:
                area_con_maximo = area
                y_maximo = posicion_area[1]
                posicion_con_maximo = posicion_area
                ultimo_simbolo = simbolo["simbolo"]
              if posicion_area[0] > x_maximo:
                x_maximo = posicion_area[0]

          # determina si entraría una temporalidad con al menos dos símbolos más        
          if (y_maximo < (posicion_limite[1]-4)) and not ("conector" in ultimo_simbolo):
            # genera temporaralidad para "resetear" diagrama
            if modo_DEBUG:
              print("\t--Resetear con temporalidad:  posicion_area: ", posicion_area, "y_maximo:", y_maximo, "posicion_limite:", posicion_limite)
            areas_generadas.remove(area_con_maximo)
            areas_generadas.append(area_con_maximo)
            ultima_coord_dibujo, origen, posicion = agregar_temporalidad(page, posicion_area, ultima_coord_dibujo)
          else:
            # no se puede agregar más símbolos
            termino = True    
            if modo_DEBUG:
              print("\t--TERMINO:  posicion: ", posicion, "y_maximo:", y_maximo, "posicion_limite:",posicion_limite)

    if (not forzar_decision and not forzar_copia and not forzar_area_externa and not forzar_actualizacion and not forzar_consulta_logica) or (forzar_decision and uso_decision) or (forzar_copia and uso_copia) or (forzar_area_externa and uso_area_externa) or (forzar_actualizacion and uso_actualizacion) or (forzar_consulta_logica and uso_consulta_logica):
      # muestra debug
      if modo_DEBUG:
        print("\t--areas_generadas: ", areas_generadas)
      # cierra el caso y lo graba 
      dibujar_lineas_areas(areas_generadas, page)
      guardar_cursograma(nombre, page, xml_annotation)
      print(" + caso guardado.")
      i += 1
    else:
      # indica por qué se descarta el caso
      if (forzar_decision and not uso_decision):
        print(" ~ descartado porque no tiene Decisión!")
      elif (forzar_copia and not uso_copia):
        print(" ~ descartado porque no usa Copias de Documentos!")
      elif (forzar_area_externa and not uso_area_externa):
        print(" ~ descartado porque no tiene Área Externa!")
      elif (forzar_actualizacion and not uso_actualizacion):
        print(" ~ descartado porque no tiene Actualización!")
      elif (forzar_consulta_logica and not uso_consulta_logica):
        print(" ~ descartado porque no tiene Consulta Lógica!")

print("\n\n** Generación Terminada **")

if forzar_drive_actualizar:
  print("\n** Forzando la actualización del drive **")
  # Fuerza la actualizacion del drive
  drive.flush_and_unmount()
  # vuelve a montar
  drive.mount('/content/gdrive', force_remount=True)
  print("**Actualización del drive terminada **")

# Muestra la cantidad de casos generados
contar_cantidad_casos(path_destino)
